# Продвинутый Python, лекция 14

**Лектор:** Петров Тимур

**Семинаристы:** Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

### Как Python работает изнутри?

### Интерпретатор

![image.png](https://kinsta.com/wp-content/uploads/2021/06/working-of-python-interpreter-1024x576.png)

Когда запускается файл с расширением `*.py` Python выполяет следующие шаги:
* компилирует программу в байт-код
* исполненяет этот байт-кода

всеми операциями заведует CPython! (по умолчанию)

например:

#### .pyc

In [ ]:
# создаем питонячий файл и вписываем туда нашу программу
!touch example.py
!echo 'print("Hello world!!!")' > example.txt


Используем [py_compile](https://docs.python.org/3/library/py_compile.html) для компиляции:



In [ ]:
import py_compile

py_compile.compile('example.py')

'__pycache__/example.cpython-310.pyc'

Когда Python импортирует модуль, он компилирует его исходный код в байт-код. Этот байт-код сохраняется в файле `.pyc` для того, чтобы его не нужно было компилировать снова при следующем запуске программы

#### .pyo

Тоже скомпилированные байт-код файлы но с дополнительной оптимизацией. Оптимизация тут в отключении assert'ов и debug отладочной информации

In [ ]:
%%writefile greet.py

def greet(name):
    assert isinstance(name, str), "Ожидаем на вход строку!"
    print(f"Hello, {name}!")

Writing greet.py


In [ ]:
!python -O -m py_compile greet.py

In [ ]:
!python -m py_compile greet.py

Декомпиляция тут пока что невозможна т.к. версия используемого интепретатора 3.12 :-(

### [dis](https://docs.python.org/3/library/dis.html)

Библиотека представляющая методы которые показывают, как Python интерпретирует наш исходный код на уровне байт-кода

Напишем простую функцию Фибоначчи и посмотрим ее байт-код

In [ ]:
def fib(n: int) -> int:
    if n <= 1:
        return n

    return fib(n - 1) + fib(n - 2)

In [ ]:
import dis

dis.dis(fib)

  1           0 RESUME                   0

  2           2 LOAD_FAST                0 (n)
              4 LOAD_CONST               1 (1)
              6 COMPARE_OP              26 (<=)
             10 POP_JUMP_IF_FALSE        2 (to 16)

  3          12 LOAD_FAST                0 (n)
             14 RETURN_VALUE

  5     >>   16 LOAD_GLOBAL              1 (NULL + fib)
             26 LOAD_FAST                0 (n)
             28 LOAD_CONST               1 (1)
             30 BINARY_OP               10 (-)
             34 CALL                     1
             42 LOAD_GLOBAL              1 (NULL + fib)
             52 LOAD_FAST                0 (n)
             54 LOAD_CONST               2 (2)
             56 BINARY_OP               10 (-)
             60 CALL                     1
             68 BINARY_OP                0 (+)
             72 RETURN_VALUE


Здесь мы видим инструкции как работает наша функция. Мы прогружаем в стэк все переменные, потом сравниваем что вызывают определенные инструкции. После выполняем 2 вызова функции (ибо у нас рекурсивные вызовы) и после уже считаем результат операциями

* LOAD_FAST загружает локальную переменную
* LOAD_CONST загружает константу
* LOAD_GLOBAL загружает глобальную переменную или функцию


* BINARY_OP стандартно исполняет математические операции над двоичными значениями
    * BINARY_SUBSTRACT или BINARY_MULTIPLY или BINARY_ADD и тд.
* INPLACE_OP стандартно исполняет над **изменением** состояния объекта, к примеру структур данных
    * INPLACE_ADD, INPLACE_SUBTRACT и тд

* CALL вызывает функцию
* RETURN_VALUE возвращает результат с функции


In [ ]:
def sum_a_b(a: int, b: int) -> int:
    return a + b

def hello_world(name : str) -> str:
    return f'Hello, world and {name}!'

In [ ]:
dis.dis(sum_a_b)

  1           0 RESUME                   0

  2           2 LOAD_FAST                0 (a)
              4 LOAD_FAST                1 (b)
              6 BINARY_OP                0 (+)
             10 RETURN_VALUE


Загружаем локальные переменные (которые находятся внутри функции) и совершаем математические операции результат сразу возвращаем

In [ ]:
dis.dis(hello_world)

  4           0 RESUME                   0

  5           2 LOAD_CONST               1 ('Hello, world and ')
              4 LOAD_FAST                0 (name)
              6 FORMAT_VALUE             0
              8 LOAD_CONST               2 ('!')
             10 BUILD_STRING             3
             12 RETURN_VALUE


F-строка состоит из константы-строки и переменной которая преобразуется в строку. Фунция преобразования `str()` обычно смотрит определенно ли у объекта `__str__` или `__repr__`, и исполняет реализацию этих dunder-методов. Если ничего не нашел, то оно кидает строку с адресом в памяти где оно находится с allias'ом место вызова

А что если у нас просто большое ветвление?

In [ ]:
from enum import Enum

class Item(Enum):
    FIRST = 'first'
    SECOND = 'second'
    THIRD = 'third'
    FOURTH = 'fourth'

def choose_item(num: int) -> str:
    match num:
        case 1:
            return Item.FIRST.value
        case 2:
            return Item.SECOND.value
        case 3:
            return Item.THIRD.value
        case 4:
            return Item.FOURTH.value

    return ''


In [ ]:
dis.dis(choose_item)

 11           0 RESUME                   0

 12           2 LOAD_FAST                0 (num)

 13           4 COPY                     1
              6 LOAD_CONST               1 (1)
              8 COMPARE_OP              40 (==)
             12 POP_JUMP_IF_FALSE       27 (to 68)
             14 POP_TOP

 14          16 LOAD_GLOBAL              0 (Item)
             26 LOAD_ATTR                2 (FIRST)
             46 LOAD_ATTR                4 (value)
             66 RETURN_VALUE

 15     >>   68 COPY                     1
             70 LOAD_CONST               2 (2)
             72 COMPARE_OP              40 (==)
             76 POP_JUMP_IF_FALSE       27 (to 132)
             78 POP_TOP

 16          80 LOAD_GLOBAL              0 (Item)
             90 LOAD_ATTR                6 (SECOND)
            110 LOAD_ATTR                4 (value)
            130 RETURN_VALUE

 17     >>  132 COPY                     1
            134 LOAD_CONST               3 (3)
            136 COMPAR

Внутри каждого блока происходит загрузка глобальной переменной Item, доступ к атрибутам (полям класса) через цепочку LOAD_ATTR, и возвращение значения через RETURN_VALUE

POP_TOP удаляет неиспользуемые значения и вызывается в нескольких случаях
* когда просто вызывается функция или метод и результат потом не используется (результат без присваивания ни к чему!)
* результат не присваивается ни к одной переменной

Давайте посмотрим:

In [ ]:
def a_in_cube(a: int) -> int:
    a ** 4 # тут должно удалиться из стека т.к нигде не используется!
    a ** 4 # тут должно удалиться из стека т.к нигде не используется!

    return a ** 3

dis.dis(a_in_cube)

  1           0 RESUME                   0

  2           2 LOAD_FAST                0 (a)
              4 LOAD_CONST               1 (4)
              6 BINARY_OP                8 (**)
             10 POP_TOP

  3          12 LOAD_FAST                0 (a)
             14 LOAD_CONST               1 (4)
             16 BINARY_OP                8 (**)
             20 POP_TOP

  5          22 LOAD_FAST                0 (a)
             24 LOAD_CONST               2 (3)
             26 BINARY_OP                8 (**)
             30 RETURN_VALUE


LOAD_ATTR достает "поля" = "атрибуты" класса
В нашем случае атрибутами были как раз `enum` объекты которые были внутри класса-энама `Item`

In [ ]:
from typing import List, Optional

class ListApi:
    def __init__(self, l: Optional[List[int]]):
        self._l = l if l is not None else []

    def get_item(self, index: int) -> int | Exception:
        try:
            return self._l[index]
        except Exception as e:
            raise e


dis.dis(ListApi)

Disassembly of __init__:
  4           0 RESUME                   0

  5           2 LOAD_FAST                1 (l)
              4 POP_JUMP_IF_NONE         8 (to 22)
              6 LOAD_FAST                1 (l)
              8 LOAD_FAST                0 (self)
             10 STORE_ATTR               0 (_l)
             20 RETURN_CONST             0 (None)
        >>   22 BUILD_LIST               0
             24 LOAD_FAST                0 (self)
             26 STORE_ATTR               0 (_l)
             36 RETURN_CONST             0 (None)

Disassembly of get_item:
  7           0 RESUME                   0

  8           2 NOP

  9           4 LOAD_FAST                0 (self)
              6 LOAD_ATTR                0 (_l)
             26 LOAD_FAST                1 (index)
             28 BINARY_SUBSCR
             32 RETURN_VALUE
        >>   34 PUSH_EXC_INFO

 10          36 LOAD_GLOBAL              2 (Exception)
             46 CHECK_EXC_MATCH
             48 POP_JUMP_IF_FA

C версии 3.11 языка, инструкция `RESUME 0` объявляется при инициализации
* Инструкции вида *%BUILD_*%* обозначают построение структур данных
* Инструкция *PUSH_EXC_INFO* перехватывает обработку исключений
* Инструкция *RERAISE* повторно выбрасывает перехваченное исключение, которое еще не обработано

### Как на счет GC (Garbage Collector)?

Как вообще работает GC в Python? Как известно, мы про него обычно никак не думаем, но тем не менее, про это надо знать.

Итак, логика максимально простая:

1. У любого объекта есть такая вещь, как счетчик ссылок. Если у объекта ссылок 0, то его пора удалять

In [10]:
import sys

a = [1,2,3,4,5,6,7,8,9,10]
print(sys.getrefcount(a)) # вопрос: а с фига ли тут 2, а не 1?
b = a
print(sys.getrefcount(a)) # оп, получили +1 ссылку

2
3


2. Данная вещь не работает, если у нас ссылки циклические. Например:

In [ ]:
class A:
    def __init__(self):
        self.b = None

class B:
    def __init__(self):
        self.a = None

a = A()
b = B()
a.b = b
b.a = a

Получается цикл (А ссылкатся на B, B ссылается на A), в котором ничего и никогда не будет удалено. Что же делать? Вот в таком случае используется уборка по генерациям (про нее позже)


Мы можем позвать его руками. \
Давайте напишем такое:

In [ ]:
import gc

class First:
    def __init__(self, a: int):
        self._a = a

    def add_number(self, b: int):
        self._a += b

    def __str__(self):
        return f'First({self._a})'

class Second:
    def __init__(self, b: int):
        self._b = b

    def add_number(self, b: int):
        # Создаем два объекта First
        first_item = First(1)
        second_item = First(2)

        self._b += first_item._a

        print("Before:")
        print(f"Collections count before GC: {gc.get_count()}")
        gc.collect()
        print("After:")
        print(f"Collections count after GC: {gc.get_count()}")

    def __str__(self):
        return f'Second({self._b})'


In [ ]:
second = Second(5)
second.add_number(10)
# нужно раскоментировать это и закомментировать вызов GC в классе Second
# gc.get_count()

Before:
Collections count before GC: (294, 1, 0)
After:
Collections count after GC: (3, 0, 0)


`gc.collect()` явно чистит неиспользуемые отслеживаемые объекты \
`gc.count()` подсчитывает количество отслеживаемых объектов \

Сборщик мусора делает это все автоматически, но явный вызов gc.collect() похож на то что вы "докопались" до еще некритичной отметки отслеживаемых объектов на которой имеется хоть одна ссылка.

### Reference Counter

Основной механизм управления памятью в Python

* Каждый объект в Python имеет связанный с ним счётчик ссылок
* Когда кто-то создаёт ссылку на объект, счётчик увеличивается.
* Когда ссылка на объект уничтожается, счётчик уменьшается.
* Когда счётчик ссылок объекта = 0, объект автоматически удаляется, и его память освобождается

### Mark-and-sweep

* В первый проход происходит маркировка всех достижимых (живых) объектов

Объекты начинают жить свою жизнь в программе

* Во втором проходе он ищет всех "немаркированных" (не живых) объектов и убивает их

### Generational Garbage Collection

Интерпретатор выделяет на 3 группы объектов существующих объектов в программе.

* Поколение 0

Недавносозданные объекты. Их держат на контроле до тех пор, пока не произойдет первый цикл сборки мусора. Обычно их маркируют отдельно т.к. частенько новосозданные объекты не используются

* Поколение 1

Новосозданный объект который пережил хотя-бы одну сборку мусора

* Поколение 2

Неуловимый Джо. Используется во всем цикле работы программы и очень редко удаляется

Давайте посмотрим на это:

In [ ]:
%%writefile gc_stats_script.py
import gc
import time

class DBConnection:
    def __init__(self, name):
        self.name = name
        print(f"Сonnected: {self.name}")

    def close(self):
        print(f"Disconnected: {self.name}")

def manage_connections():
    # Создаем 6 соединений для "количества"
    connections = [DBConnection(f"conn{i}") for i in range(6)]

    # Ожидаем, чтобы они не были удалены слишком быстро
    time.sleep(4)

    for conn in connections:
        conn.close()

    # Удаляем все эти соединения
    for conn in connections:
        del conn

    # Пауза, чтобы некоторые объекты смогли попасть в старшие поколения.
    time.sleep(3)

    print("After cycle:")
    print(f"Count of generation 0: {gc.get_count()[0]}")
    print(f"Count of generation 1: {gc.get_count()[1]}")
    print(f"Count of generation 2: {gc.get_count()[2]}")

# "Активируем" GC
gc.enable()

# Инициализируем новые соединения
manage_connections()

# Принудительно сбрасываем мусор!
gc.collect()


Overwriting gc_stats_script.py


In [ ]:
!python3 gc_stats_script.py

Сonnected: conn0
Сonnected: conn1
Сonnected: conn2
Сonnected: conn3
Сonnected: conn4
Сonnected: conn5
Disconnected: conn0
Disconnected: conn1
Disconnected: conn2
Disconnected: conn3
Disconnected: conn4
Disconnected: conn5
After cycle:
Count of generation 0: 684
Count of generation 1: 4
Count of generation 2: 0


### Name resolution

#### LEGB

* **L**ocal -- имена, определенные в текущем теле (фукнции)
* **E**nclosing -- имена, определенные в функциях, которые находятся в области видимости текущей функции
* **G**lobal -- имена, определенные в глобальной области видимости
* **B**uiltin -- имена, определенные в встроенной области видимости `(print, len, ...)`



In [ ]:
x = "Global"

def enclosing_function():
    x = "Global local"
    def local_function():
        x = "Local"
        print(x)

    local_function()

 # Где Python будет искать переменную x?

enclosing_function()

Local


### Type-hints

Типизация в языке пока что остается неявной. Напрямую интепретатор никак не взаимодействует с ней, так что это пока остается на уровне "глаз". Но в основном как это работает:

In [ ]:
def sample_func(a: int, b: str) -> None:
    a += 1
    b = b[:2]

In [ ]:
print(sample_func.__annotations__)

{'a': <class 'int'>, 'b': <class 'str'>, 'return': None}


После интепретации кода, все типы остается в метаданных, которые получить можно через dunder-метод `__annotations__`

## Животное дня

![](https://habrastorage.org/getpro/habr/upload_files/175/98b/1f6/17598b1f6212efb2eef05f5487413bbb.jpeg)

И это свинья! К сожалению, про них обычно говорят в негативной коннотации, но это грустно, конечно, давайте немного про них:

1. Едят все подряд - ну по факту это фича, а не бага. Больше источников еды - больше возможность выжить (люди тоже являются всеядными, если что)

2. Катаются в грязи - а это абсолютно нормальное поведение для любых животных с очень мелких прошерстком, грязь позволяет меньше страдать от жары (что, например, делают те же самые слоны), защищаться от солнечных лучей. В целом тоже фича

И в целом негативное отношение к свинье идет больше из религиозных побуждений, чем каких-то рациональных (в языческом мире свиней часто ассоциировали с плодородием). А вообще свиньи - это не только мясо, но и спасение для человечества: за счет того, что достаточное число органов очень похожи на человеческие (например, кожа, сердце, глаза), что позволяет на них обучаться будущим медикам и проводить на них испытания.

А также по уровню интеллекта, свиньи могут быть не глупее собак (вероятно, что свиньи умеют проходить MSR (mirror self-recognition) тест)